<a href="https://colab.research.google.com/github/BoredBolby/TheStudentLot/blob/main/TheStudentLot_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Hi Kolby#

In [3]:
!pip install requests pandas

In [6]:
import requests
import pandas as pd
# --- Configuration ---
# 1. Paste your API Key inside the quotes below
API_KEY = "LpI5P0VnbYzpwzTmyiCnJnbjunJfIaOjTqmklPspGhdez//JNSe/XqDasQArCcyN"
#2 This run will pull stats for 2024, Week 1 to test
YEAR = 2024
WEEK = 1

# --- SETUP ---
BASE_URL = "https://api.collegefootballdata.com"
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json"
}

# --- THE REQUEST ---
print(f"Attempting to fetch data for {YEAR} WEEk {WEEK}...")

# Using the "Player Game Stats" endpoint
# This gives total yards/TD per player

response = requests.get(
    f"{BASE_URL}/games/players",
    headers=headers,
    params={"year": YEAR, "week": WEEK}
)

# --- PROCESSING ---
if response.status_code == 200:
  data = response.json()
  print(f"Success! Downloaded stats for {len(data)} games.")

  # data comes in nested (Game -> Team -> Player -> Stat)
  #need to flatten into rows and columns like a spreadsheet

  records = []

  for game in data:
    game_id = game['id']
    for team in game['teams']:
      team_name = team['school']

      # Loop through passing, rushing, receiving categories

      # Passing
      for category in team['categories']:
        stat_type = category['passing'] #e.g., "passing", "rushing"
        for player in category['types']:
          stat_name = player['passing'] #e.g. "passing", "rushing"
          for athlete in player['athletes']:
            records.append({"Player": athlete['name'],
                            "Team": team_name,
                            "GameID": game_id,
                            "Stat Type": stat_name,
                            "Value": athlete['stat']
                            })
        stat_type = category['rushing']
        for player in category['types']:
          stat_name = player['rushing']
          for athlete in player['athletes']:
            records.append({"Player": athlete['name'],
                            "Team": team_name,
                            "GameID": game_id,
                            "Stat Type": stat_name,
                            "Value": athlete['stat']
                            })
        stat_type = category['receiving']
        for player in category['types']:
          stat_name = player['receiving']
          for athlete in player['athletes']:
            records.append({"Player": athlete['name'],
                            "Team": team_name,
                            "GameID": game_id,
                            "Stat Type": stat_name,
                            "Value": athlete['stat']
                            })


      # add rushing and receiving --------------------------------
  # Convert to a Pandas DataFrame
  df = pd.DataFrame(records)

  # Show the first 5 rows to verify
  print("\nHere is a preview of your data:")
  display(df.head())


Attempting to fetch data for 2024 WEEk 1...
Success! Downloaded stats for 187 games.


KeyError: 'school'